# Limits
First, we establish a rate limiter for Binance. This will avoid overcharging the rate limiter and potentially hitting the rate limit that can ban your account.

In [1]:
from panzer.limits import BinanceRateLimiter

In [ ]:
l = BinanceRateLimiter()

In [ ]:
l.get()

# Errors

In [ ]:
from panzer.errors import BinanceAPIException
from panzer.request import get

try:
    # missing symbol example
    response = get(url="https://api.binance.com/api/v3/klines", params=[("interval", "1m"),])

except BinanceAPIException as e:
    print(f"Error: {e}")

# Credentials
First, we add api keys and secret to the credential manager. If not, they will be asked by prompt. Adding with is_sensitive parameter will save the credentials securely. If not 
provided then it will be automatically secured if name of the credential is "api_key" or "api_secret".

In [ ]:
from panzer.keys import CredentialManager

In [ ]:
c = CredentialManager()

In [ ]:
c.credentials

## Cached credentials
If CredentialManager is asked for a credential that is already in the cache, it will return the cached value instead of asking for it again. Even it is not loaded in the object 
at first creation, it will add from disk cache automatically when needed, and it will prompt for it if it is not loaded. 

In [ ]:
c.add("api_key", "test", is_sensitive=True, overwrite=False)
c.add("api_secret", "test", is_sensitive=True, overwrite=False)

In [ ]:
c.get("api_secret", decrypt=True)

In [ ]:
# renew to load real creds from file
c = CredentialManager()

In [ ]:
# cached in file can be retrieved with `file_manager` attribute.
# c.file_manager

In [ ]:
c

# Examples of API calls
Credentials for API calls are going to be asked by prompt the first time if they are not in the creds cache.


In [ ]:
from panzer.request import get, post

## Get klines example
Assuming credentials are already in the cache, will check limits and then make the API call.

In [ ]:
url = 'https://api.binance.com/api/v3/klines'
weight=2
params = {
    "symbol": "BTCUSDT",  # Par BTCUSDT
    "interval": "1m",     # Intervalo de 1 minuto
    "limit": 3            # Limitar a las últimas 5 velas
}

In [ ]:
l.can_make_request(url=url, params_qty=len(params), weight=weight, is_order=False)

In [ ]:
response, headers = get(params=params, 
                        url=url,
                        full_sign=False,
                        server_time_offset=l.server_time_offset,)

In [ ]:
# first limits update will notify sync of the object with api server if they differ
l.update_from_headers(url=url, params_qty=len(params), headers=headers, expected_weight=weight)

In [ ]:
response

In [ ]:
headers

### Without weight
It can get weight for url from a local database created upon use of the module. Also is_order can be inferred from the url.

In [ ]:
l.can_make_request(url=url, params_qty=len(params), weight=None, is_order=None)
response, headers = get(params=params, 
                        url=url,
                        full_sign=False,
                        server_time_offset=l.server_time_offset,)
l.update_from_headers(url=url, params_qty=len(params), headers=headers, expected_weight=weight)

## Test order

In [ ]:
url = 'https://api.binance.com/api/v3/order/test'
weight = 1

# timestamp is automatically added when signed call
params = {'symbol': "BTCUSDT",
          'side': "SELL",
          'type': "LIMIT",
          'timeInForce': 'GTC',
          'quantity': 0.001,
          'price': 80000,
          'recvWindow': 10000}

In [ ]:
if l.can_make_request(url=url, params_qty=len(params), weight=weight, is_order=False):
    response, headers = post(params=params, 
                            url=url,
                            full_sign=True,
                            server_time_offset=l.server_time_offset,)
l.update_from_headers(url=url, params_qty=len(params), headers=headers, expected_weight=weight)

In [ ]:
response

In [ ]:
headers

## Trades

In [ ]:
url = 'https://api.binance.com/api/v3/myTrades'
weight = 20
params = {
    'symbol': 'BTCUSDT',                   # The trading pair
    'limit': 3,                            # Optional: Limit the number of trades to retrieve (default 500)
    'recvWindow': 5000                     # Optional: Time window for the request (default 5000 ms)
}

In [ ]:
if l.can_make_request(url=url, params_qty=len(params), weight=weight, is_order=False):
    response, headers = get(params=params, 
                            url=url,
                            full_sign=True,
                            server_time_offset=l.server_time_offset,)
l.update_from_headers(url=url, params_qty=len(params), headers=headers, expected_weight=weight)

In [ ]:
response

In [ ]:
headers

In [ ]:
l.get()